# Q1

In [1]:
import json
import requests
import pandas as pd

In [2]:
def create_destinations_df(api_key, origin, destinations_file):
    # Read in destinations from file
    with open(destinations_file, 'r') as f:
        destinations = f.read().split('\n')

    # Initialize DataFrame
    df = pd.DataFrame(columns=['Target', 'Distance_km', 'Duration (hour+minutes)', 'Longitude', 'Latitude'])

    # Get distance and duration
    for destination in destinations:
        # Get distance and duration from Google Distance Matrix API
        url = "https://maps.googleapis.com/maps/api/distancematrix/json?origins=%s&destinations=%s&key=%s" % (origin, destination, api_key)
        try:
            response = requests.get(url)
            if not response.status_code == 200:
                print("HTTP error", response.status_code)
            else:
                response_data = response.json()
                distance_km = response_data['rows'][0]['elements'][0]['distance']['value'] / 1000
                duration_sec = response_data['rows'][0]['elements'][0]['duration']['value']
                duration_hours = duration_sec // 3600
                duration_minutes = (duration_sec % 3600) // 60
                duration_str = f"{duration_hours}h {duration_minutes}m"
        except requests.exceptions.RequestException as e:
            print("Error making request:", e)

        # Get latitude and longitude from Google Geocoding API
        url = "https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (destination, api_key)
        try:
            response = requests.get(url)
            if not response.status_code == 200:
                print("HTTP error", response.status_code)
            else:
                response_data = response.json()
                lat_lng = response_data['results'][0]['geometry']['location']
                latitude = lat_lng['lat']
                longitude = lat_lng['lng']
        except requests.exceptions.RequestException as e:
            print("Error making request:", e)

        # Add row to DataFrame --> with 'pd.concat' instead of 'df.append', because this method will be removed from pandas in a future version
        new_row = {'Target': destination,
                   'Distance_km': distance_km,
                   'Duration (hour+minutes)': duration_str,
                   'Longitude': longitude,
                   'Latitude': latitude}
        df = pd.concat([df, pd.DataFrame(new_row, index=[0])], ignore_index=True)


    return df


In [3]:
api_key = "API_KEY"
origin = "Tel Aviv, Israel"
destinations_file = "dests.txt"

# Q2

In [4]:
df = create_destinations_df(api_key, origin, destinations_file)
df

,Target,Distance_km,Duration (hour+minutes),Longitude,Latitude
0,Istanbul,1815.227,21h 2m,28.978359,41.008238
1,Amsterdam,4532.541,48h 3m,4.904139,52.367573
2,Valletta,3792.883,50h 50m,14.514100,35.899237
3,Basel,4092.805,44h 2m,7.588576,47.559599
4,Doha,2164.477,22h 38m,51.531040,25.285447


# Q3

In [5]:
df_sorted = df.sort_values(by='Distance_km', ascending=False)
furthest_cities = df_sorted.head(3)['Target']
furthest_cities

1    Amsterdam
3        Basel
2     Valletta
Name: Target, dtype: object